In [ ]:
!pip install
requests
pandas
datetime
ccxt
ta
numpy

ERROR: Could not find a version that satisfies the requirement py_option_pricing (from versions: none)
ERROR: No matching distribution found for py_option_pricing


In [ ]:
#COIN GLASS KEY

API_KEY = '936395e1aa6943659c5ff7b729981532'

# chave API da Santiment
API_s = 'eo2w54kemtpvjfkn_mbevy33adklhwvcx'

In [2]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import ccxt
import numpy as np
import ta
import pytz
import yfinance as yf
import matplotlib.pyplot as plt
from fredapi import Fred
from bs4 import BeautifulSoup

In [ ]:
##################################################################################### cvd #######################################################
def fetch_trades(pair='XXBTZUSD', interval=30, lookback_days=14):
    # Função para obter dados da API (Exemplo com Kraken)
    endpoint = 'https://api.kraken.com/0/public/OHLC'
    end_time = datetime.utcnow()
    start_time = end_time - timedelta(days=lookback_days)
    start_time_ts = int(start_time.timestamp())

    params = {
        'pair': pair,
        'interval': interval,
        'since': start_time_ts
    }

    response = requests.get(endpoint, params=params)
    data = response.json()

    if 'result' not in data:
        raise ValueError("Não foi possível obter os dados da API.")

    ohlc_data = data['result'].get(pair, [])
    columns = ['time', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'count']
    df = pd.DataFrame(ohlc_data, columns=columns)

    # Converte timestamps para datetime e ajusta para o horário de São Paulo (BRT)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df['time'] = df['time'].dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')
    df['time'] = df['time'].dt.tz_localize(None)

    df['volume'] = pd.to_numeric(df['volume'], errors='coerce')

    return df

def calculate_cvd_30min(df):
    # Calcular o CVD para cada janela de 30 minutos separadamente
    df.set_index('time', inplace=True)  # Definindo o índice como o tempo
    df = df.resample('30T').agg({'volume': 'sum'})

    # Define o CVD como o volume total por cada intervalo de 30 minutos
    df['cvd'] = df['volume']
    df.reset_index(inplace=True)  # Redefinindo o índice para manter a coluna de tempo

    return df[['time', 'cvd']]

df = fetch_trades()

cvd_30min = calculate_cvd_30min(df)

print(cvd_30min)


                   time        cvd
0   2024-08-31 03:30:00   1.072680
1   2024-08-31 04:00:00   3.021198
2   2024-08-31 04:30:00   6.764411
3   2024-08-31 05:00:00   0.486285
4   2024-08-31 05:30:00   4.012819
..                  ...        ...
661 2024-09-13 22:00:00  51.852455
662 2024-09-13 22:30:00  52.471886
663 2024-09-13 23:00:00  12.187186
664 2024-09-13 23:30:00  22.301140
665 2024-09-14 00:00:00  35.611899

[666 rows x 2 columns]


C:\Users\Francisco\AppData\Local\Temp\ipykernel_19588\1147374816.py:37: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df = df.resample('30T').agg({'volume': 'sum'})


In [ ]:
################################################################################      MUDANCA DE VOLUME      ################################################################################

# Função para obter dados históricos do Bitcoin
def puxar_dados():
    url = 'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart'
    params = {
        'vs_currency': 'usd',
        'days': '14',  # últimos 14
        'interval': 'daily'
    }
    response = requests.get(url, params=params)
    data = response.json()

    # Extraindo dados relevantes
    prices = data['prices']
    volumes = data['total_volumes']

    # Convertendo para DataFrame
    df_prices = pd.DataFrame(prices, columns=['timestamp', 'price'])
    df_prices['timestamp'] = pd.to_datetime(df_prices['timestamp'], unit='ms')

    df_volumes = pd.DataFrame(volumes, columns=['timestamp', 'volume'])
    df_volumes['timestamp'] = pd.to_datetime(df_volumes['timestamp'], unit='ms')

    # Merge prices and volumes
    df = pd.merge(df_prices, df_volumes, on='timestamp')

    return df

df = puxar_dados()

# Calculando a mudança no volume (como proxy para open interest)
df['change_in_volume'] = df['volume'].diff()

# Armazenando a mudança no volume em uma variável
mudanca_volume = df[['timestamp', 'change_in_volume']]

# Exibindo o resultado
print(mudanca_volume)


             timestamp  change_in_volume
0  2024-08-30 00:00:00               NaN
1  2024-08-31 00:00:00      1.147516e+10
2  2024-09-01 00:00:00     -3.263941e+10
3  2024-09-02 00:00:00      1.406654e+10
4  2024-09-03 00:00:00      2.547160e+09
5  2024-09-04 00:00:00     -1.182799e+09
6  2024-09-05 00:00:00      1.072040e+10
7  2024-09-06 00:00:00     -7.544607e+09
8  2024-09-07 00:00:00      2.025504e+10
9  2024-09-08 00:00:00     -3.321237e+10
10 2024-09-09 00:00:00      1.596375e+09
11 2024-09-10 00:00:00      1.701060e+10
12 2024-09-11 00:00:00     -5.979326e+09
13 2024-09-12 00:00:00      8.089559e+09
14 2024-09-12 21:27:37     -3.710449e+09


In [ ]:
########################################################################### OPEN INTEREST ##########################################################

import requests
import pandas as pd
from datetime import datetime, timedelta

def get_ohlc_kraken(pair='XXBTZUSD', interval=30, lookback_days=30):
    endpoint = 'https://api.kraken.com/0/public/OHLC'

    end_time = datetime.utcnow()
    start_time = end_time - timedelta(days=lookback_days)

    start_time_ts = int(start_time.timestamp())

    params = {
        'pair': pair,
        'interval': interval,
        'since': start_time_ts
    }

    response = requests.get(endpoint, params=params)

    if response.status_code == 200:
        data = response.json()
        if 'result' in data:
            ohlc_data = data['result'][pair]

            if ohlc_data and len(ohlc_data) > 0:
                # Define as colunas
                num_columns = len(ohlc_data[0])
                columns = ['time', 'open', 'high', 'low', 'close', 'volume', 'count', 'extra'][:num_columns]

                # Converte para DataFrame
                df = pd.DataFrame(ohlc_data, columns=columns)

                # Converte timestamps para datetime
                df['time'] = pd.to_datetime(df['time'], unit='s')

                # Ajusta o tempo para o horário de São Paulo (BRT)
                df['time'] = df['time'].dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')

                # Se o 'volume' está presente, usa como open interest
                if 'volume' in df.columns:
                    df['date'] = df['time'].dt.date
                    df['time_of_day'] = df['time'].dt.time  # Renomeia para evitar conflito com a coluna 'time'
                    df.rename(columns={'volume': 'open_interest'}, inplace=True)
                    result_df = df[['date', 'time_of_day', 'open_interest']]
                    return result_df
                else:
                    raise ValueError("Volume column not found in the data.")
            else:
                raise ValueError("No data found for the given parameters.")
        else:
            raise Exception("Unexpected response format or no data found.")
    else:
        raise Exception(f"Error fetching data: {response.status_code}")

pair = 'XXBTZUSD'
open_interest = get_ohlc_kraken(pair)

print(open_interest)




           date time_of_day open_interest
0    2024-08-29    11:00:00       60695.3
1    2024-08-29    11:30:00       60836.3
2    2024-08-29    12:00:00       60945.0
3    2024-08-29    12:30:00       60953.3
4    2024-08-29    13:00:00       60757.7
..          ...         ...           ...
715  2024-09-13    08:30:00       58002.7
716  2024-09-13    09:00:00       57830.0
717  2024-09-13    09:30:00       57728.8
718  2024-09-13    10:00:00       57775.7
719  2024-09-13    10:30:00       57868.7

[720 rows x 3 columns]


In [ ]:
################################################################################################# SKEW #####################################################################################

# Endpoint da API da Deribit para dados de opções de Bitcoin
def get_options_skew():
    url = 'https://www.deribit.com/api/v2/public/get_book_summary_by_currency?currency=BTC&kind=option'
    response = requests.get(url)

    if response.status_code == 200:
        options_data = response.json().get('result', [])
        if not options_data:
            print("Nenhum dado de opções foi retornado.")
            return None

        df = pd.DataFrame(options_data)


        # Cálculo de skew usando os preços de ask e bid
        if 'ask_price' in df.columns and 'bid_price' in df.columns:
            df['skew'] = (df['ask_price'] - df['bid_price']) / df['mid_price'] * 100  # Skew como a diferença percentual
            return df[['instrument_name', 'ask_price', 'bid_price', 'mid_price', 'skew']]
        else:
            print("As colunas de preço (ask_price e bid_price) não estão disponíveis nos dados.")
            return None

    else:
        print("Erro ao obter dados de opções:", response.status_code, response.text)
        return None

# Chamada da função
skew = get_options_skew()
print(skew)


         instrument_name  ask_price  bid_price  mid_price       skew
0    BTC-27SEP24-22000-C     0.6205     0.6175    0.61900   0.484653
1    BTC-12SEP24-53500-C     0.0740     0.0700    0.07200   5.555556
2    BTC-25OCT24-47000-P     0.0155     0.0145    0.01500   6.666667
3    BTC-27DEC24-15000-C     0.7475     0.7435    0.74550   0.536553
4    BTC-25OCT24-57000-C     0.0855     0.0845    0.08500   1.176471
..                   ...        ...        ...        ...        ...
831  BTC-12SEP24-55000-C     0.0500     0.0460    0.04800   8.333333
832  BTC-29NOV24-52000-P     0.0630     0.0615    0.06225   2.409639
833  BTC-12SEP24-63000-C     0.0005     0.0003    0.00040  50.000000
834  BTC-29NOV24-35000-C     0.4160     0.4075    0.41175   2.064359
835  BTC-13SEP24-90000-C     0.0001        NaN        NaN        NaN

[836 rows x 5 columns]


In [ ]:
######################################################################################################## iv #############################################################

# Endpoint da API da Deribit para dados de opções de Bitcoin
def get_options_iv():
    url = 'https://www.deribit.com/api/v2/public/get_book_summary_by_currency?currency=BTC&kind=option'
    response = requests.get(url)

    if response.status_code == 200:
        options_data = response.json().get('result', [])
        if not options_data:
            print("Nenhum dado de opções foi retornado.")
            return None

        df = pd.DataFrame(options_data)

        # Verifique se o campo de volatilidade implícita está disponível
        if 'mark_iv' in df.columns:
            # Filtrar apenas as colunas relevantes para volatilidade implícita
            df_iv = df[['instrument_name', 'mark_iv']]
            return df_iv
        else:
            print("O campo de volatilidade implícita (mark_iv) não está disponível nos dados.")
            return None

    else:
        print("Erro ao obter dados de opções:", response.status_code, response.text)
        return None

# Chamada da função
iv = get_options_iv()
print(iv)


         instrument_name  mark_iv
0    BTC-27SEP24-22000-C   135.22
1    BTC-12SEP24-53500-C    82.86
2    BTC-25OCT24-47000-P    59.81
3    BTC-27DEC24-15000-C    96.55
4    BTC-25OCT24-57000-C    54.84
..                   ...      ...
831  BTC-12SEP24-55000-C    74.56
832  BTC-29NOV24-52000-P    62.46
833  BTC-12SEP24-63000-C    73.27
834  BTC-29NOV24-35000-C    71.94
835  BTC-13SEP24-90000-C    95.12

[836 rows x 2 columns]


In [ ]:
############################################################################################### VOLUME DE NEGOCIAÇÃO ###################################################################
# Endpoint da API da Deribit para dados de opções de Bitcoin
def get_options_volume():
    url = 'https://www.deribit.com/api/v2/public/get_book_summary_by_currency?currency=BTC&kind=option'
    response = requests.get(url)

    if response.status_code == 200:
        options_data = response.json().get('result', [])
        if not options_data:
            print("Nenhum dado de opções foi retornado.")
            return None

        df = pd.DataFrame(options_data)

        # Verifique se o campo de volume de negociação está disponível
        if 'volume' in df.columns:
            # Filtrar apenas as colunas relevantes para o volume de negociação
            df_volume = df[['instrument_name', 'volume']]
            return df_volume
        else:
            print("O campo de volume de negociação (volume) não está disponível nos dados.")
            return None

    else:
        print("Erro ao obter dados de opções:", response.status_code, response.text)
        return None

# Chamada da função
vol_negociacao = get_options_volume()

print(vol_negociacao)


         instrument_name  volume
0    BTC-27SEP24-22000-C     0.0
1    BTC-12SEP24-53500-C     0.0
2    BTC-25OCT24-47000-P    12.2
3    BTC-27DEC24-15000-C     0.0
4    BTC-25OCT24-57000-C     1.1
..                   ...     ...
831  BTC-12SEP24-55000-C     2.1
832  BTC-29NOV24-52000-P    14.7
833  BTC-12SEP24-63000-C     5.3
834  BTC-29NOV24-35000-C     0.0
835  BTC-13SEP24-90000-C     0.0

[836 rows x 2 columns]


In [ ]:
############################################################################### profundidade do mercado ######################################################
# Configurações gerais
API_KEY = '936395e1aa6943659c5ff7b729981532'
BASE_URL = "https://open-api-v3.coinglass.com/api/futures/orderbook/history"
headers = {
    "accept": "application/json",
    "CG-API-KEY": API_KEY
}

# Função para capturar a profundidade do mercado (Order Book) de 30 em 30 minutos
def get_order_book_depth(symbol, exchange="Binance", interval="30m", limit=1000, start_time=None, end_time=None):
    params = {
        "exchange": exchange,
        "symbol": symbol,
        "interval": interval,
        "limit": limit,
        "startTime": start_time,
        "endTime": end_time
    }
    response = requests.get(BASE_URL, headers=headers, params=params)
    print("Resposta da API:", response.status_code)  # Verificar status da requisição
    if response.status_code != 200:
        print(f"Erro na requisição: {response.text}")
        return None
    data = response.json()
    print("Dados retornados pela API:", data)  # Imprimir os dados retornados
    return data

# Função para converter timestamp para data e hora de São Paulo
def convert_to_brt(timestamp):
    dt = datetime.utcfromtimestamp(timestamp)
    brt_tz = pytz.timezone('America/Sao_Paulo')
    dt_brt = pytz.utc.localize(dt).astimezone(brt_tz)
    return dt_brt.strftime('%Y-%m-%d %H:%M:%S')

# Função para processar os dados da profundidade do mercado e armazenar em um DataFrame
def process_order_book_to_df(data):
    if data is None:
        return pd.DataFrame()  # Retornar DataFrame vazio em caso de erro na API
    records = []
    for item in data.get("data", []):
        bids_usd = item.get("bidsUsd", "N/A")
        bids_amount = item.get("bidsAmount", "N/A")
        asks_usd = item.get("asksUsd", "N/A")
        asks_amount = item.get("asksAmount", "N/A")
        timestamp = item.get("time", 0)
        # Converter timestamp para data e hora em São Paulo
        brt_time = convert_to_brt(timestamp)
        records.append({
            "Data/Hora (São Paulo)": brt_time,
            "Bids (USD)": bids_usd,
            "Bids Amount": bids_amount,
            "Asks (USD)": asks_usd,
            "Asks Amount": asks_amount
        })
    df = pd.DataFrame(records)
    return df

# Função para capturar dados em blocos de 30 dias (30 minutos de intervalo por requisição)
def fetch_order_book_in_blocks(symbol, interval="30m", days=30):
    now = datetime.utcnow()
    df_list = []

    while days > 0:
        end_time = int(now.timestamp())
        start_time = int((now - timedelta(minutes=30 * 1000)).timestamp())  # 1000 intervalos de 30 minutos
        order_book_data = get_order_book_depth(symbol, start_time=start_time, end_time=end_time, interval=interval)
        df = process_order_book_to_df(order_book_data)
        if df.empty:
            print("Nenhum dado retornado para este intervalo.")
            break
        df_list.append(df)

        # Atualiza a variável 'now' para continuar capturando períodos anteriores
        now = now - timedelta(minutes=30 * 1000)
        days -= (30 * 1000) / (24 * 60)  # Subtrai o número de dias correspondentes

    # Concatena todos os DataFrames
    final_df = pd.concat(df_list, ignore_index=True)
    return final_df

# Capturar a profundidade do mercado para o par BTCUSDT nos últimos 30 dias de 30 em 30 minutos
symbol = "BTCUSDT"
profudidade = fetch_order_book_in_blocks(symbol)
print(profudidade)


Resposta da API: 200
Dados retornados pela API: {'code': '0', 'msg': 'success', 'data': [{'bidsUsd': 168038526.4513, 'bidsAmount': 2650.33, 'asksUsd': 84500963.7206, 'asksAmount': 1319.036, 'time': 1724463000}, {'bidsUsd': 136026710.3179, 'bidsAmount': 2145.169, 'asksUsd': 89126553.8479, 'asksAmount': 1389.855, 'time': 1724464800}, {'bidsUsd': 133150198.2475, 'bidsAmount': 2098.247, 'asksUsd': 96761275.0151, 'asksAmount': 1508.228, 'time': 1724466600}, {'bidsUsd': 109865982.1823, 'bidsAmount': 1727.543, 'asksUsd': 107721401.6777, 'asksAmount': 1677.628, 'time': 1724468400}, {'bidsUsd': 98215834.9478, 'bidsAmount': 1545.976, 'asksUsd': 110639086.8019, 'asksAmount': 1723.865, 'time': 1724470200}, {'bidsUsd': 104486394.7342, 'bidsAmount': 1646.869, 'asksUsd': 107201024.7828, 'asksAmount': 1672.801, 'time': 1724472000}, {'bidsUsd': 104088540.3154, 'bidsAmount': 1638.07, 'asksUsd': 105992854.3535, 'asksAmount': 1651.595, 'time': 1724473800}, {'bidsUsd': 128122959.2072, 'bidsAmount': 2017.22

In [ ]:
##################################################################################### OLHC DERIVATIVO ############################################

# Configurar a URL da API com o intervalo de 30 minutos
url = "https://open-api-v3.coinglass.com/api/futures/openInterest/ohlc-history"
params = {
    'exchange': 'Binance',   # Nome da exchange
    'symbol': 'BTCUSDT',     # Símbolo do par de negociação
    'interval': '30m'        # Intervalo de 30 minutos
}

# Cabeçalhos da requisição com a chave da API
headers = {
    "accept": "application/json",
    "CG-API-KEY": API_KEY  # Substitua pela sua chave de API
}

# Fazer a requisição à API
response = requests.get(url, headers=headers, params=params)

# Verificar se a resposta é bem-sucedida
if response.status_code == 200:
    data = response.json()

    if data['code'] == "0":
        # Normalizar os dados JSON para um DataFrame
        df = pd.DataFrame(data['data'])

        # Converter o timestamp (t) para o formato de data legível no horário de São Paulo
        df['time'] = pd.to_datetime(df['t'], unit='s')
        df['time'] = df['time'].dt.tz_localize('UTC').dt.tz_convert('America/Sao_Paulo')

        # Renomear colunas para facilitar a leitura
        df.rename(columns={'o': 'open', 'h': 'high', 'l': 'low', 'c': 'close'}, inplace=True)

        # Selecionar apenas as colunas desejadas
        olhc = df[['time', 'open', 'high', 'low', 'close']]

        # Exibir o DataFrame
        print(olhc)

    else:
        print(f"Erro na resposta da API: {data['msg']}")
else:
    print(f"Erro ao fazer a requisição: {response.status_code} - {response.text}")


                         time      open       high       low      close
0   2024-08-23 15:30:00-03:00   88735.6   89211.71   88717.9   89211.71
1   2024-08-23 16:00:00-03:00  89211.71    89462.3   88902.9    88902.9
2   2024-08-23 16:30:00-03:00   88902.9   88966.07  88673.79   88758.03
3   2024-08-23 17:00:00-03:00  88758.03   89425.73  88758.03   89425.73
4   2024-08-23 17:30:00-03:00  89425.73   89839.59  89404.39   89839.59
..                        ...       ...        ...       ...        ...
995 2024-09-13 09:00:00-03:00  87062.76   87497.06  86976.69    87398.5
996 2024-09-13 09:30:00-03:00   87398.5    87553.9  87210.67   87231.78
997 2024-09-13 10:00:00-03:00  87231.78   87309.37  87214.73   87266.24
998 2024-09-13 10:30:00-03:00  87266.24   87704.33  87266.24   87290.32
999 2024-09-13 11:00:00-03:00  87290.32  87291.883  87248.99  87291.883

[1000 rows x 5 columns]


In [ ]:
##################################################################################### liquidacoes de long / short ###############################################################################
# Configurações gerais

BASE_URL = "https://open-api-v3.coinglass.com/api/futures/liquidation/v2/history"
headers = {
    "accept": "application/json",
    "CG-API-KEY": API_KEY
}

# Função para capturar o histórico de liquidações de Long/Short de 30 em 30 minutos
def get_liquidation_history(symbol, exchange="Binance", interval="30m", limit=1000, start_time=None, end_time=None):
    params = {
        "exchange": exchange,
        "symbol": symbol,
        "interval": interval,
        "limit": limit,
        "startTime": start_time,
        "endTime": end_time
    }
    response = requests.get(BASE_URL, headers=headers, params=params)
    return response.json()

# Função para converter timestamp para data e hora de São Paulo
def convert_to_brt(timestamp):
    dt = datetime.utcfromtimestamp(timestamp)
    brt_tz = pytz.timezone('America/Sao_Paulo')
    dt_brt = pytz.utc.localize(dt).astimezone(brt_tz)
    return dt_brt.strftime('%Y-%m-%d %H:%M:%S')

# Função para processar os dados e armazenar em um DataFrame
def process_liquidations_to_df(data):
    records = []
    for item in data.get("data", []):
        long_liq = item.get("longLiquidationUsd", "N/A")
        short_liq = item.get("shortLiquidationUsd", "N/A")
        timestamp = item.get("t", 0)
        brt_time = convert_to_brt(timestamp)
        records.append({
            "Data/Hora (São Paulo)": brt_time,
            "Long Liquidation (USD)": long_liq,
            "Short Liquidation (USD)": short_liq
        })
    df = pd.DataFrame(records)
    return df

# Função para capturar dados em blocos de 30 dias (30 minutos de intervalo por requisição)
def fetch_data_in_blocks(symbol, interval="30m", days=30):
    now = datetime.utcnow()
    df_list = []

    # Dividimos os 30 dias em blocos de intervalos de 1000 registros (aprox. 20.8 dias de 30 minutos)
    while days > 0:
        end_time = int(now.timestamp())
        start_time = int((now - timedelta(minutes=30 * 1000)).timestamp())  # 1000 intervalos de 30 minutos
        liquidation_data = get_liquidation_history(symbol, start_time=start_time, end_time=end_time, interval=interval)
        df = process_liquidations_to_df(liquidation_data)
        df_list.append(df)

        # Atualiza a variável 'now' para continuar capturando períodos anteriores
        now = now - timedelta(minutes=30 * 1000)
        days -= (30 * 1000) / (24 * 60)  # Subtrai o número de dias correspondentes

    # Concatena todos os DataFrames
    final_df = pd.concat(df_list, ignore_index=True)
    return final_df

# Capturar liquidações para o par BTCUSDT nos últimos 30 dias de 30 em 30 minutos
symbol = "BTCUSDT"
liquidacoes = fetch_data_in_blocks(symbol)

print(liquidacoes)



     Data/Hora (São Paulo) Long Liquidation (USD) Short Liquidation (USD)
0      2024-08-23 21:00:00            16299.96707             409482.0944
1      2024-08-23 21:30:00             7419.27275             107838.0065
2      2024-08-23 22:00:00             296938.126               12836.834
3      2024-08-23 22:30:00              115794.56                127.6998
4      2024-08-23 23:00:00                      0              54464.5094
...                    ...                    ...                     ...
1989   2024-08-23 18:30:00              2245.6719              152383.731
1990   2024-08-23 19:00:00              9173.5834             202463.2784
1991   2024-08-23 19:30:00           116108.80973              88597.2397
1992   2024-08-23 20:00:00               49894.95                257.1968
1993   2024-08-23 20:30:00           250559.59034                 896.532

[1994 rows x 3 columns]


In [ ]:
######################################################################################### long_short_ratio #########################################################

BASE_URL = "https://open-api-v3.coinglass.com/api/futures/globalLongShortAccountRatio/history"
headers = {
    "accept": "application/json",
    "CG-API-KEY": API_KEY
}

# Função para converter timestamp para data e hora de São Paulo
def convert_to_brt(timestamp):
    dt = datetime.utcfromtimestamp(timestamp)
    brt_tz = pytz.timezone('America/Sao_Paulo')
    dt_brt = pytz.utc.localize(dt).astimezone(brt_tz)
    return dt_brt.strftime('%Y-%m-%d %H:%M:%S')

# Função para capturar o Long/Short Ratio de 30 em 30 minutos
def get_long_short_ratio(symbol, exchange="Binance", interval="30m", limit=1000, start_time=None, end_time=None):
    params = {
        "exchange": exchange,
        "symbol": symbol,
        "interval": interval,
        "limit": limit,
        "startTime": start_time,
        "endTime": end_time
    }
    response = requests.get(BASE_URL, headers=headers, params=params)
    return response.json()

# Função para processar os dados e armazenar em um DataFrame
def process_long_short_ratio_to_df(data):
    records = []
    for item in data.get("data", []):
        long_account = item.get("longAccount", "N/A")
        short_account = item.get("shortAccount", "N/A")
        long_short_ratio = item.get("longShortRatio", "N/A")
        timestamp = item.get("time", 0)
        # Converter timestamp para data e hora em São Paulo
        brt_time = convert_to_brt(timestamp)
        records.append({
            "Data/Hora (São Paulo)": brt_time,
            "Long Account (%)": long_account,
            "Short Account (%)": short_account,
            "Long/Short Ratio": long_short_ratio
        })
    df = pd.DataFrame(records)
    return df

# Função para capturar dados em blocos de 1000 registros (aprox. 20.8 dias de 30 minutos por requisição)
def fetch_data_in_blocks(symbol, interval="30m", total_days=30):
    now = datetime.utcnow()
    df_list = []

    while total_days > 0:
        end_time = int(now.timestamp())
        start_time = int((now - timedelta(minutes=30 * 1000)).timestamp())  # 1000 intervalos de 30 minutos

        ratio_data = get_long_short_ratio(symbol, start_time=start_time, end_time=end_time, interval=interval)
        df = process_long_short_ratio_to_df(ratio_data)
        df_list.append(df)

        # Atualiza a variável 'now' para continuar capturando períodos anteriores
        now = now - timedelta(minutes=30 * 1000)
        total_days -= (30 * 1000) / (24 * 60)  # Subtrai o número de dias correspondentes

    # Concatena todos os DataFrames
    final_df = pd.concat(df_list, ignore_index=True)
    return final_df

# Capturar Long/Short Ratio para o par BTCUSDT nos últimos 30 dias de 30 em 30 minutos
symbol = "BTCUSDT"
long_short_ratio = fetch_data_in_blocks(symbol)

# Exibir o DataFrame
print(long_short_ratio)


     Data/Hora (São Paulo)  Long Account (%)  Short Account (%)  \
0      2024-08-23 23:00:00             47.01              52.99   
1      2024-08-23 23:30:00             47.30              52.70   
2      2024-08-24 00:00:00             47.40              52.60   
3      2024-08-24 00:30:00             47.47              52.53   
4      2024-08-24 01:00:00             47.26              52.74   
...                    ...               ...                ...   
1989   2024-08-23 20:30:00             48.04              51.96   
1990   2024-08-23 21:00:00             47.45              52.55   
1991   2024-08-23 21:30:00             47.35              52.65   
1992   2024-08-23 22:00:00             47.57              52.43   
1993   2024-08-23 22:30:00             46.95              53.05   

      Long/Short Ratio  
0                0.887  
1                0.898  
2                0.901  
3                0.904  
4                0.896  
...                ...  
1989             0.9

In [ ]:
############################################################################################ FUNDING RATE OHLC c#################################################


BASE_URL = "https://open-api-v3.coinglass.com/api/futures/fundingRate/ohlc-history"
headers = {
    "accept": "application/json",
    "CG-API-KEY": API_KEY
}

# Função para converter timestamp para data e hora de São Paulo
def convert_to_brt(timestamp):
    dt = datetime.utcfromtimestamp(timestamp)
    brt_tz = pytz.timezone('America/Sao_Paulo')
    dt_brt = pytz.utc.localize(dt).astimezone(brt_tz)
    return dt_brt.strftime('%Y-%m-%d %H:%M:%S')

# Função para obter o timestamp Unix
def get_unix_timestamp(days_ago=0):
    dt = datetime.now() - timedelta(days=days_ago)
    return int(dt.timestamp())

# Função para capturar OHLC History (Funding Rate) dos últimos 30 dias com intervalos de 30 minutos
def get_funding_rate_ohlc(symbol="BTCUSDT", exchange="Binance", interval="30m"):
    # Definir o período dos últimos 30 dias
    end_time = get_unix_timestamp(0)  # Agora
    start_time = get_unix_timestamp(30)  # Últimos 30 dias

    params = {
        "exchange": exchange,
        "symbol": symbol,
        "interval": interval,
        "startTime": start_time,
        "endTime": end_time
    }

    response = requests.get(BASE_URL, headers=headers, params=params)

    # Verificar status da resposta
    print("Status da resposta:", response.status_code)

    if response.status_code != 200:
        print("Erro na requisição:", response.text)
        return pd.DataFrame()

    data = response.json()

    if not data.get("data"):
        print("Nenhum dado retornado pela API.")
        return pd.DataFrame()

    # Processar os dados
    records = []
    for item in data.get("data", []):
        open_price = item.get("o", "N/A")
        high_price = item.get("h", "N/A")
        low_price = item.get("l", "N/A")
        close_price = item.get("c", "N/A")
        timestamp = item.get("t", 0)
        brt_time = convert_to_brt(timestamp)
        records.append({
            "Data/Hora (São Paulo)": brt_time,
            "Open": open_price,
            "High": high_price,
            "Low": low_price,
            "Close": close_price
        })

    # Criar DataFrame
    df = pd.DataFrame(records)
    return df

# Capturar OHLC History para o par BTCUSDT com intervalos de 30 minutos nos últimos 30 dias
funding_rate_ohlc = get_funding_rate_ohlc(interval="30m")
print(funding_rate_ohlc)


Status da resposta: 200
     Data/Hora (São Paulo)       Open       High        Low      Close
0      2024-08-14 16:00:00  -0.009925  -0.009869  -0.010199  -0.010188
1      2024-08-14 16:30:00  -0.010188  -0.010072  -0.010248  -0.010083
2      2024-08-14 17:00:00  -0.010083  -0.010061  -0.010304  -0.010304
3      2024-08-14 17:30:00  -0.010304  -0.010052  -0.010335  -0.010052
4      2024-08-14 18:00:00  -0.010052   -0.00955  -0.010133   -0.00955
...                    ...        ...        ...        ...        ...
1435   2024-09-13 13:30:00  -0.003073  -0.003073  -0.003576  -0.003149
1436   2024-09-13 14:00:00  -0.003149  -0.002933  -0.003164   -0.00295
1437   2024-09-13 14:30:00   -0.00295  -0.002795  -0.003138  -0.003138
1438   2024-09-13 15:00:00  -0.003138  -0.003138  -0.003531  -0.003531
1439   2024-09-13 15:30:00  -0.003531  -0.003531  -0.003877  -0.003877

[1440 rows x 5 columns]


In [ ]:
############################################################################################ FUNDING RATE oi_weight_ohlc c#################################################


BASE_URL = "https://open-api-v3.coinglass.com/api/futures/fundingRate/oi-weight-ohlc-history"
headers = {
    "accept": "application/json",
    "CG-API-KEY": API_KEY
}

# Função para converter timestamp para data e hora de São Paulo
def convert_to_brt(timestamp):
    dt = datetime.utcfromtimestamp(timestamp)
    brt_tz = pytz.timezone('America/Sao_Paulo')
    dt_brt = pytz.utc.localize(dt).astimezone(brt_tz)
    return dt_brt.strftime('%Y-%m-%d %H:%M:%S')

# Função para obter o timestamp Unix
def get_unix_timestamp(days_ago=0):
    dt = datetime.now() - timedelta(days=days_ago)
    return int(dt.timestamp())

# Função para capturar OI Weight OHLC History dos últimos 30 dias com intervalos de 30 minutos
def get_oi_weight_ohlc_history(symbol="BTC", interval="30m"):
    # Definir o período dos últimos 30 dias
    end_time = get_unix_timestamp(0)  # Agora
    start_time = get_unix_timestamp(30)  # Últimos 30 dias

    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": start_time,
        "endTime": end_time
    }

    response = requests.get(BASE_URL, headers=headers, params=params)

    # Verificar status da resposta
    print("Status da resposta:", response.status_code)

    if response.status_code != 200:
        print("Erro na requisição:", response.text)
        return pd.DataFrame()

    data = response.json()

    if not data.get("data"):
        print("Nenhum dado retornado pela API.")
        return pd.DataFrame()

    # Processar os dados
    records = []
    for item in data.get("data", []):
        open_price = item.get("o", "N/A")
        high_price = item.get("h", "N/A")
        low_price = item.get("l", "N/A")
        close_price = item.get("c", "N/A")
        timestamp = item.get("t", 0)
        brt_time = convert_to_brt(timestamp)
        records.append({
            "Data/Hora (São Paulo)": brt_time,
            "Open": open_price,
            "High": high_price,
            "Low": low_price,
            "Close": close_price
        })

    # Criar DataFrame
    df = pd.DataFrame(records)
    return df

# Capturar OI Weight OHLC History para o par BTC com intervalos de 30 minutos nos últimos 30 dias
oi_weight_ohlc = get_oi_weight_ohlc_history(interval="30m")
print(oi_weight_ohlc)


Status da resposta: 200
     Data/Hora (São Paulo)       Open       High        Low      Close
0      2024-08-14 16:00:00  -0.005706  -0.004326  -0.005706   -0.00451
1      2024-08-14 16:30:00   -0.00451  -0.003667  -0.004672  -0.003667
2      2024-08-14 17:00:00  -0.003667  -0.003065  -0.003681  -0.003443
3      2024-08-14 17:30:00  -0.003443  -0.002936  -0.003458  -0.003013
4      2024-08-14 18:00:00  -0.003013  -0.001824  -0.003013  -0.001883
...                    ...        ...        ...        ...        ...
1435   2024-09-13 13:30:00  -0.003942  -0.003296  -0.004286  -0.003702
1436   2024-09-13 14:00:00  -0.003702  -0.003034  -0.003761  -0.003139
1437   2024-09-13 14:30:00  -0.003139  -0.003118  -0.004455  -0.004455
1438   2024-09-13 15:00:00  -0.004455  -0.004455   -0.00573   -0.00573
1439   2024-09-13 15:30:00   -0.00573  -0.005632  -0.005872  -0.005635

[1440 rows x 5 columns]


In [ ]:
############################################################################################ FUNDING RATE vol weight ohl c#################################################

BASE_URL = "https://open-api-v3.coinglass.com/api/futures/fundingRate/vol-weight-ohlc-history"
headers = {
    "accept": "application/json",
    "CG-API-KEY": API_KEY
}

# Função para converter timestamp para data e hora de São Paulo
def convert_to_brt(timestamp):
    dt = datetime.utcfromtimestamp(timestamp)
    brt_tz = pytz.timezone('America/Sao_Paulo')
    dt_brt = pytz.utc.localize(dt).astimezone(brt_tz)
    return dt_brt.strftime('%Y-%m-%d %H:%M:%S')

# Função para obter o timestamp Unix
def get_unix_timestamp(days_ago=0):
    dt = datetime.now() - timedelta(days=days_ago)
    return int(dt.timestamp())

# Função para capturar Vol Weight OHLC History dos últimos 30 dias com intervalos de 30 minutos
def get_vol_weight_ohlc_history(symbol="BTC", interval="30m"):
    # Definir o período dos últimos 30 dias
    end_time = get_unix_timestamp(0)  # Agora
    start_time = get_unix_timestamp(30)  # Últimos 30 dias

    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": start_time,
        "endTime": end_time
    }

    response = requests.get(BASE_URL, headers=headers, params=params)

    # Verificar status da resposta
    print("Status da resposta:", response.status_code)

    if response.status_code != 200:
        print("Erro na requisição:", response.text)
        return pd.DataFrame()

    data = response.json()

    if not data.get("data"):
        print("Nenhum dado retornado pela API.")
        return pd.DataFrame()

    # Processar os dados
    records = []
    for item in data.get("data", []):
        open_price = item.get("o", "N/A")
        high_price = item.get("h", "N/A")
        low_price = item.get("l", "N/A")
        close_price = item.get("c", "N/A")
        timestamp = item.get("t", 0)
        brt_time = convert_to_brt(timestamp)
        records.append({
            "Data/Hora (São Paulo)": brt_time,
            "Open": open_price,
            "High": high_price,
            "Low": low_price,
            "Close": close_price
        })

    # Criar DataFrame
    df = pd.DataFrame(records)
    return df

# Capturar Vol Weight OHLC History para o par BTC com intervalo de 30 minutos nos últimos 30 dias
vol_weight_ohlc = get_vol_weight_ohlc_history(interval="30m")
print(vol_weight_ohlc)


Status da resposta: 200
     Data/Hora (São Paulo)       Open       High        Low      Close
0      2024-08-14 16:00:00  -0.007713  -0.006436  -0.007775  -0.006764
1      2024-08-14 16:30:00  -0.006764  -0.006107  -0.007052  -0.006107
2      2024-08-14 17:00:00  -0.006107   -0.00527  -0.006118  -0.005727
3      2024-08-14 17:30:00  -0.005727  -0.005065  -0.005766   -0.00529
4      2024-08-14 18:00:00   -0.00529  -0.004057   -0.00529  -0.004191
...                    ...        ...        ...        ...        ...
1435   2024-09-13 13:30:00  -0.004055  -0.004055  -0.004729  -0.004605
1436   2024-09-13 14:00:00  -0.004605  -0.004087  -0.004723  -0.004125
1437   2024-09-13 14:30:00  -0.004125  -0.004092   -0.00494   -0.00494
1438   2024-09-13 15:00:00   -0.00494   -0.00494  -0.006097  -0.006097
1439   2024-09-13 15:30:00  -0.006097  -0.006097  -0.006097  -0.006097

[1440 rows x 5 columns]


In [ ]:
########################################################################################### open_interest_ohlc derivativo ############################################################################
BASE_URL = "https://open-api-v3.coinglass.com/api/futures/openInterest/ohlc-history"
headers = {
    "accept": "application/json",
    "CG-API-KEY": API_KEY
}

# Função para converter timestamp para data e hora de São Paulo
def convert_to_brt(timestamp):
    dt = datetime.utcfromtimestamp(timestamp)
    brt_tz = pytz.timezone('America/Sao_Paulo')
    dt_brt = pytz.utc.localize(dt).astimezone(brt_tz)
    return dt_brt.strftime('%Y-%m-%d %H:%M:%S')

# Função para obter o timestamp Unix
def get_unix_timestamp(days_ago=0):
    dt = datetime.now() - timedelta(days=days_ago)
    return int(dt.timestamp())

# Função para capturar Open Interest OHLC History dos últimos 30 dias com intervalos de 30 minutos
def get_open_interest_ohlc_history(symbol="BTCUSDT", exchange="Binance", interval="30m"):
    # Definir o período dos últimos 30 dias
    end_time = get_unix_timestamp(0)
    start_time = get_unix_timestamp(30)  # Últimos 30 dias

    params = {
        "exchange": exchange,
        "symbol": symbol,
        "interval": interval,
        "startTime": start_time,
        "endTime": end_time
    }

    response = requests.get(BASE_URL, headers=headers, params=params)

    # Verificar status da resposta
    print("Status da resposta:", response.status_code)

    if response.status_code != 200:
        print("Erro na requisição:", response.text)
        return pd.DataFrame()

    data = response.json()

    if not data.get("data"):
        print("Nenhum dado retornado pela API.")
        return pd.DataFrame()

    # Processar os dados
    records = []
    for item in data.get("data", []):
        open_price = item.get("o", "N/A")
        high_price = item.get("h", "N/A")
        low_price = item.get("l", "N/A")
        close_price = item.get("c", "N/A")
        timestamp = item.get("t", 0)
        brt_time = convert_to_brt(timestamp)
        records.append({
            "Data/Hora (São Paulo)": brt_time,
            "Open": open_price,
            "High": high_price,
            "Low": low_price,
            "Close": close_price
        })

    # Criar DataFrame
    df = pd.DataFrame(records)
    return df

# Capturar Open Interest OHLC History para o par BTCUSDT com intervalos de 30 minutos nos últimos 30 dias
open_interest_ohlc = get_open_interest_ohlc_history(interval="30m")
print(open_interest_ohlc)


Status da resposta: 200
     Data/Hora (São Paulo)      Open       High       Low      Close
0      2024-08-14 15:30:00  78761.02   78773.62  78222.89   78271.27
1      2024-08-14 16:00:00  78271.27   78306.56  78125.97   78303.42
2      2024-08-14 16:30:00  78303.42   78423.42  78303.42   78420.76
3      2024-08-14 17:00:00  78420.76   78508.61  78409.07   78420.16
4      2024-08-14 17:30:00  78420.16   78453.11  78182.07   78182.07
...                    ...       ...        ...       ...        ...
1435   2024-09-13 13:00:00  87977.23   89063.02  87977.23   89058.28
1436   2024-09-13 13:30:00  89058.28   89068.08  88861.44   88929.93
1437   2024-09-13 14:00:00  88929.93   89019.51  88848.42   89019.51
1438   2024-09-13 14:30:00  89019.51   89150.18  88235.13    88450.2
1439   2024-09-13 15:00:00   88450.2  88607.436  88330.83  88607.436

[1440 rows x 5 columns]


In [ ]:
################################################################################################## open interest ohlc_aggregated_history ###############################################################################
# Configurações gerais
BASE_URL = "https://open-api-v3.coinglass.com/api/futures/openInterest/ohlc-aggregated-history"
headers = {
    "accept": "application/json",
    "CG-API-KEY": API_KEY
}

# Função para converter timestamp para data e hora de São Paulo
def convert_to_brt(timestamp):
    dt = datetime.utcfromtimestamp(timestamp)
    brt_tz = pytz.timezone('America/Sao_Paulo')
    dt_brt = pytz.utc.localize(dt).astimezone(brt_tz)
    return dt_brt.strftime('%Y-%m-%d %H:%M:%S')

# Função para obter o timestamp Unix
def get_unix_timestamp(days_ago=0):
    dt = datetime.now() - timedelta(days=days_ago)
    return int(dt.timestamp())

# Função para capturar OHLC Aggregated History dos últimos 30 dias com intervalos de 30 minutos
def get_ohlc_aggregated_history(symbol="BTC", interval="30m"):
    # Definir o período dos últimos 30 dias
    end_time = get_unix_timestamp(0)
    start_time = get_unix_timestamp(30)  # Últimos 30 dias

    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": start_time,
        "endTime": end_time
    }

    response = requests.get(BASE_URL, headers=headers, params=params)

    # Verificar status da resposta
    print("Status da resposta:", response.status_code)

    if response.status_code != 200:
        print("Erro na requisição:", response.text)
        return pd.DataFrame()

    data = response.json()

    if not data.get("data"):
        print("Nenhum dado retornado pela API.")
        return pd.DataFrame()

    # Processar os dados
    records = []
    for item in data.get("data", []):
        open_price = item.get("o", "N/A")
        high_price = item.get("h", "N/A")
        low_price = item.get("l", "N/A")
        close_price = item.get("c", "N/A")
        timestamp = item.get("t", 0)
        brt_time = convert_to_brt(timestamp)
        records.append({
            "Data/Hora (São Paulo)": brt_time,
            "Open": open_price,
            "High": high_price,
            "Low": low_price,
            "Close": close_price
        })

    # Criar DataFrame
    df = pd.DataFrame(records)
    return df

# Capturar OHLC Aggregated History para o par BTC com intervalos de 30 minutos nos últimos 30 dias
ohlc_aggregated_history = get_ohlc_aggregated_history(interval="30m")
print(ohlc_aggregated_history)


Status da resposta: 200
     Data/Hora (São Paulo)              Open              High  \
0      2024-08-14 16:00:00       29449831501       29515107044   
1      2024-08-14 16:30:00       29479978314       29536179131   
2      2024-08-14 17:00:00       29497016813       29597364884   
3      2024-08-14 17:30:00       29543956236       29562036363   
4      2024-08-14 18:00:00       29539452605       29539452605   
...                    ...               ...               ...   
1435   2024-09-13 13:30:00       31366483262       31376171112   
1436   2024-09-13 14:00:00       31209088977       31352336874   
1437   2024-09-13 14:30:00       31334104563       31470537019   
1438   2024-09-13 15:00:00       31236922668       31289678099   
1439   2024-09-13 15:30:00  31224173071.5874  31224173071.5874   

                   Low             Close  
0          29395834650       29479978314  
1          29463353396       29497016813  
2          29497016813       29543956236  
3          

In [ ]:
############################################################################### volume de transações on-chain ####################################################################

# URL da API para obter o volume de transações on-chain para o Bitcoin
url = f'https://api.santiment.net/graphql'

# Ajustar as datas para dentro do intervalo permitido (14 de setembro de 2023 em diante)
to_date = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")
from_date = "2023-09-14T00:00:00Z"  # Começo permitido

# Query ajustada com as datas permitidas
query = f"""
{{
  getMetric(metric: "transaction_volume") {{
    timeseriesData(
      slug: "bitcoin"
      from: "{from_date}"
      to: "{to_date}"
      interval: "30m"
    ) {{
      datetime
      value
    }}
  }}
}}
"""

# Fazendo a requisição para a API da Santiment
response = requests.post(url, json={'query': query}, headers={'Authorization': f'Apikey {API_s}'})

# Verificando se a requisição foi bem-sucedida
if response.status_code == 200:
    # Exibindo o JSON retornado para verificar a estrutura
    data = response.json()
    print(data)  # Exibe a resposta JSON completa para análise

    # Acessando os dados corretos
    data = data['data']['getMetric']['timeseriesData']

    # Convertendo os dados para DataFrame do pandas
    if data:
        df = pd.DataFrame(data)

        # Verifique se o nome da coluna de tempo é 'datetime' ou outro
        if 'datetime' in df.columns:
            df['datetime'] = pd.to_datetime(df['datetime'])
        else:
            print("A chave 'datetime' não foi encontrada. Verifique o JSON retornado.")
            print(df.columns)  # Exibe os nomes das colunas para ajudar a encontrar o correto

        # Ajustando o fuso horário para Brasília (BRT)
        df['datetime'] = pd.to_datetime(df['datetime']).dt.tz_convert('America/Sao_Paulo')

        # Exibindo o DataFrame final
        print(df)
    else:
        print("Nenhum dado foi retornado para o período solicitado.")
else:
    print(f'Erro na requisição: {response.status_code}')


{'data': {'getMetric': {'timeseriesData': [{'datetime': '2023-09-14T20:30:00Z', 'value': 1575.412319400003}, {'datetime': '2023-09-14T21:00:00Z', 'value': 4021.54724382998}, {'datetime': '2023-09-14T21:30:00Z', 'value': 2691.090738210002}, {'datetime': '2023-09-14T22:00:00Z', 'value': 3287.070255450002}, {'datetime': '2023-09-14T22:30:00Z', 'value': 4562.221049890004}, {'datetime': '2023-09-14T23:00:00Z', 'value': 2878.707607950021}, {'datetime': '2023-09-14T23:30:00Z', 'value': 2021.325543050019}, {'datetime': '2023-09-15T00:00:00Z', 'value': 3145.928332300002}, {'datetime': '2023-09-15T00:30:00Z', 'value': 4441.201370740092}, {'datetime': '2023-09-15T01:00:00Z', 'value': 2763.041905300021}, {'datetime': '2023-09-15T01:30:00Z', 'value': 1995.371669380001}, {'datetime': '2023-09-15T02:00:00Z', 'value': 6060.448473480052}, {'datetime': '2023-09-15T02:30:00Z', 'value': 3856.270049990004}, {'datetime': '2023-09-15T03:00:00Z', 'value': 4876.395655350022}, {'datetime': '2023-09-15T03:30:00Z

In [8]:
############################################################# 2 ultimas tx de juros #############################################



# Substitua 'SUA_API_KEY' pela sua chave de API do FRED
fred = Fred(api_key='1771455ee342bfff3fbe3f616c497951')

# Obter a série de dados da taxa de juros federal (Federal Funds Rate)
data = fred.get_series('FEDFUNDS')

# Capturar as duas últimas taxas
ultimas_duas_taxas = data.tail(2)
print("As duas últimas taxas de juros do Federal Reserve:")
print(ultimas_duas_taxas)



As duas últimas taxas de juros do Federal Reserve:
2024-07-01    5.33
2024-08-01    5.33
dtype: float64


In [10]:
################################################ CPI###############################################################

# Sua chave da API FRED (você precisa se registrar no FRED e obter uma chave de API)
API_KEY = '1771455ee342bfff3fbe3f616c497951'

# Inicializa a API FRED
fred = Fred(api_key=API_KEY)

# Captura a série temporal do CPI para os EUA (indicador: 'CPIAUCSL')
cpi_data = fred.get_series('CPIAUCSL')

# Armazena os dois últimos valores do CPI em variáveis
cpi_latest = cpi_data.iloc[-1]  # Valor mais recente
cpi_previous = cpi_data.iloc[-2]  # Valor anterior

print(f"Valor mais recente do CPI (Fed): {cpi_latest}")
print(f"Valor anterior do CPI (Fed): {cpi_previous}")

Valor mais recente do CPI (Fed): 314.121
Valor anterior do CPI (Fed): 313.534


In [11]:
###################################################### PCE ##################################################

# Inicializa a API FRED
fred = Fred(api_key=API_KEY)

# Captura a série temporal do PCE para os EUA (indicador: 'PCE')
pce_data = fred.get_series('PCE')

# Armazena os dois últimos valores do PCE em variáveis
pce_latest = pce_data.iloc[-1]  # Valor mais recente
pce_previous = pce_data.iloc[-2]  # Valor anterior

print(f"Valor mais recente do PCE (Fed): {pce_latest}")
print(f"Valor anterior do PCE (Fed): {pce_previous}")

Valor mais recente do PCE (Fed): 19580.5
Valor anterior do PCE (Fed): 19476.7


In [16]:
################################################################## pib ########################################################################
# Função para capturar os dois últimos PIBs
def obter_dois_ultimos_pibs():
    # Captura os dados do PIB dos EUA
    pib_series = fred.get_series('GDPC1')  # 'GDPC1' é a série para PIB real dos EUA

    # Pegue os últimos 2 valores de PIB (trimestrais)
    pib_ultimos_dois = pib_series.tail(2)

    return pib_ultimos_dois

# Executa a função e guarda os resultados
pib_ultimos_dois = obter_dois_ultimos_pibs()

# Exibindo os dados capturados
print(pib_ultimos_dois)






2024-01-01    22758.752
2024-04-01    22924.863
dtype: float64


In [26]:
################################################################## indice de ações ###################################################
# Função para gerar a análise com base nos dados de um índice
def gerar_analise(data, nome_indice):
    # Coletar os dados mais recentes
    ultimo_dado = data.tail(1)

    # Extração dos valores relevantes
    close = ultimo_dado['Close'].values[0]
    ma20 = ultimo_dado['MA20'].values[0]
    rsi = ultimo_dado['RSI'].values[0]
    bb_upper = ultimo_dado['BB_Upper'].values[0]
    bb_lower = ultimo_dado['BB_Lower'].values[0]

    # Análise do movimento de preço com base na MA20
    if close > ma20:
        tendencia = "alta"
    elif close < ma20:
        tendencia = "baixa"
    else:
        tendencia = "lateralizado"

    # Análise do RSI
    if rsi < 30:
        rsi_analise = "sobrevendido"
    elif rsi > 70:
        rsi_analise = "sobrecomprado"
    else:
        rsi_analise = "neutro"

    # Análise da posição do preço em relação às Bandas de Bollinger
    if close > bb_upper:
        bollinger_analise = "rompeu a banda superior"
    elif close < bb_lower:
        bollinger_analise = "rompeu a banda inferior"
    else:
        bollinger_analise = "dentro das bandas"

    # Montar a análise textual
    analise = f"""
    O índice {nome_indice} está atualmente em uma tendência de {tendencia}, com o preço de fechamento em {close:.2f} e a média móvel de 20 dias em {ma20:.2f}.
    O RSI indica que o mercado está {rsi_analise} com um valor de {rsi:.2f}.
    O preço está {bollinger_analise}, o que sugere que o mercado está {('em possível continuidade da tendência de alta' if bollinger_analise == 'rompeu a banda superior' else 'dentro de condições normais')}.
    """
    return analise

# Análise para o S&P 500
analise_sp500_texto = gerar_analise(analise_sp500, "S&P 500")

# Análise para o Nasdaq
analise_nasdaq_texto = gerar_analise(analise_nasdaq, "Nasdaq")

# Exibir as análises
print("Análise S&P 500:")
print(analise_sp500_texto)

print("\nAnálise Nasdaq:")
print(analise_nasdaq_texto)


Análise S&P 500:

    O índice S&P 500 está atualmente em uma tendência de alta, com o preço de fechamento em 5626.02 e a média móvel de 20 dias em 5568.21.
    O RSI indica que o mercado está neutro com um valor de 49.25.
    O preço está dentro das bandas, o que sugere que o mercado está dentro de condições normais.
    

Análise Nasdaq:

    O índice Nasdaq está atualmente em uma tendência de alta, com o preço de fechamento em 17683.98 e a média móvel de 20 dias em 17480.35.
    O RSI indica que o mercado está neutro com um valor de 46.44.
    O preço está dentro das bandas, o que sugere que o mercado está dentro de condições normais.
    


In [6]:
import requests
import pandas as pd

# Sua chave de API da NewsAPI
api_key = "d2fc198a1d2244edbdaa7ec51fd0e191"

# Função para buscar notícias da API
def get_news(query, api_key):
    url = f"https://newsapi.org/v2/everything?q={query}&from=2024-08-14&sortBy=publishedAt&apiKey={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Verifica se a requisição foi bem-sucedida
        data = response.json()

        # Verificar se há artigos nas notícias retornadas
        if data["status"] == "ok" and data["totalResults"] > 0:
            return data["articles"]
        else:
            return []

    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar notícias: {e}")
        return []

# Buscar notícias geopolíticas e sobre Bitcoin
geopolitical_news = get_news("geopolitical events", api_key)
bitcoin_news = get_news("bitcoin", api_key)

# Converter as notícias em DataFrames
def create_news_df(news_list):
    if len(news_list) > 0:
        df = pd.DataFrame(news_list)
        df = df[['source', 'author', 'title', 'description', 'url', 'publishedAt']]  # Seleciona colunas relevantes
        df['source'] = df['source'].apply(lambda x: x['name'])  # Extrair nome da fonte
        return df
    else:
        return pd.DataFrame()  # Retorna um DataFrame vazio se não houver notícias

# Criar DataFrames para as notícias
geopolitical_news_df = create_news_df(geopolitical_news)
bitcoin_news_df = create_news_df(bitcoin_news)

geopolitical_news_df.to_csv('teste.csv')

                        source                  author  \
0            Globalresearch.ca            Drago Bosnic   
1            Project Syndicate             Adam Nelson   
2                GlobeNewswire               VEON Ltd.   
3            Project Syndicate            Erik Berglöf   
4              Hospitality Net          Douglas Louden   
..                         ...                     ...   
95                Ritholtz.com          Barry Ritholtz   
96               GlobeNewswire  Globaltech Corporation   
97  Columbia Journalism Review              Kevin Lind   
98               Investing.com           Investing.com   
99         Nakedcapitalism.com         Nick Corbishley   

                                                title  \
0   Russian Hypersonic Dominance Results in Hundre...   
1   From the Year of Elections to the Year of Gove...   
2   VEON General Counsel Omiyinka Doris Named Amon...   
3                   Africa’s Path to Green Prosperity   
4    Singapore vs 

In [5]:
######################################################## noticias geopoliticas ################################################
# Sua chave de API da NewsAPI
api_key = "d2fc198a1d2244edbdaa7ec51fd0e191"

# Função para buscar notícias da API
def get_news(query, api_key):
    url = f"https://newsapi.org/v2/everything?q={query}&from=2024-08-14&sortBy=publishedAt&apiKey={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Verifica se a requisição foi bem-sucedida
        data = response.json()

        # Verificar se há artigos nas notícias retornadas
        if data["status"] == "ok" and data["totalResults"] > 0:
            return data["articles"]
        else:
            return []

    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar notícias: {e}")
        return []

# Buscar notícias geopolíticas e sobre Bitcoin
geopolitical_news = get_news("geopolitical events", api_key)
bitcoin_news = get_news("bitcoin", api_key)

# Exibir as notícias capturadas
print("Notícias Geopolíticas:")
for i, article in enumerate(geopolitical_news, start=1):
    print(f"{i}. {article['title']}")
    print(f"Resumo: {article['description']}")
    print(f"Link: {article['url']}\n")

print("\nNotícias sobre Bitcoin:")
for i, article in enumerate(bitcoin_news, start=1):
    print(f"{i}. {article['title']}")
    print(f"Resumo: {article['description']}")
    print(f"Link: {article['url']}\n")


Notícias Geopolíticas:
1. Russian Hypersonic Dominance Results in Hundreds of NATO Casualties in Ukraine
Resumo: As the entire US-led political West continues to lag decades behind Russia in hypersonic weapons, it keeps using various coping mechanisms to “explain” the gap. 
This ranges from rather pathetic claims that Moscow “stole American hypersonic technologies” (the…
Link: https://www.globalresearch.ca/russian-hypersonic-hundreds-nato-casualties-ukraine/5867165

2. From the Year of Elections to the Year of Governance
Resumo: The biggest election year in history has already led to significant political shifts around the world, underscoring the resilience of democratic systems. But to minimize the risk of political backsliding, newly elected reformist leaders must deliver on their …
Link: https://www.project-syndicate.org/commentary/five-key-challenges-facing-newly-elected-democratic-reformers-by-adam-nelson-and-kristen-sample-2024-09

3. VEON General Counsel Omiyinka Doris Named Amo

In [30]:
################################################################## correl ouro e btc ####################################

# Função para baixar os dados
def get_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    return data['Close']

# Definir o período de análise (últimos 30 dias a partir da data atual)
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=30)).strftime('%Y-%m-%d')

# Baixar dados do BTC e do ouro (XAU/USD)
btc_data = get_data('BTC-USD', start_date, end_date)
gold_data = get_data('GC=F', start_date, end_date)  # GC=F é o ticker do ouro na yfinance

# Combinar os dados em um DataFrame
df = pd.DataFrame({'BTC': btc_data, 'Gold': gold_data})

# Remover linhas com dados faltantes
df = df.dropna()

# Calcular a correlação entre BTC e ouro
correlacao_xau_btc = df.corr().loc['BTC', 'Gold']
print(f"Correlação entre BTC e Ouro nos últimos 30 dias: {correlacao_xau_btc}")



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Correlação entre BTC e Ouro nos últimos 30 dias: 0.23977834052066013
